<a href="https://colab.research.google.com/github/juhee3199/Machine-learning_advanced-study/blob/master/classification/Ensemble-Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 기본 스태킹 모델

- 위스콘신 암 데이터 셋 적용

In [ ]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import  load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

x_data = cancer_data.data
y_label = cancer_data.target

x_train,x_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.2, random_state=0)

In [ ]:
# 개별 모델
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)
# 최종 학습 모델
lr_final = LogisticRegression(C=10)

#개별 모델 학습
knn_clf.fit(x_train,y_train)
rf_clf.fit(x_train,y_train)
dt_clf.fit(x_train,y_train)
ada_clf.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [ ]:
# 개별 모델 예측
knn_pred = knn_clf.predict(x_test)
rf_pred = rf_clf.predict(x_test)
dt_pred = dt_clf.predict(x_test)
ada_pred = ada_clf.predict(x_test)

print('knn 정확도: {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정트리 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('AdaBoost 정확도: {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

knn 정확도: 0.9211
랜덤 포레스트 정확도: 0.9649
결정트리 정확도: 0.9123
AdaBoost 정확도: 0.9561


In [ ]:
# 최종 메타 모델의 학습 데이터 셋 구성  
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)
# 행과 열의 위치를 교환하여 각 알고리즘의 예측 결과를 하나의 피처로 만듬
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [ ]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델 정확도: {0:.4f}'.format(accuracy_score(y_test, final)))

최종 메타 모델 정확도: 0.9649


## 2. CV 세트 기반의 스태킹

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 1. 개별 모델로 메타 모델을 위한 학습/테스트 데이터를 생성한다.
def get_stacking_base_datasets(model, x_train_n, y_train_n, x_test_n, n_folds ):
  # kfold 생성
  kf = KFold(n_splits = n_folds, shuffle = False, random_state=0)
  # 추후에 메타 모델이 사용할 학습 데이터 반환을 위해 넘파이 배열 초기화
  train_fold_pred = np.zeros((x_train_n.shape[0],1))
  test_pred = np.zeros((x_test_n.shape[0], n_folds))
  print(model.__class__.__name__, ' model 시작 ')

  # 원본 학습 데이터를 N개의 폴드로 나눈다.
  for folder_counter, (train_index, valid_index) in enumerate(kf.split(x_train_n)):
    # 개별 모델이 학습/예측할 폴드 데이터 셋 추출
    print('\t 폴드 세트: ', folder_counter, ' 시작 ')
    x_tr = x_train_n[train_index]
    y_tr = y_train_n[train_index]
    x_te = x_train_n[valid_index]

    # 폴드 세트 내부에서 만들어진 training folds로 개별 모델의 학습 수행
    model.fit(x_tr, y_tr)
    # 폴드 세트 내부에서 만들어진 validation folds로 개별 모델을 통해 예측 후 데이터 저장
    train_fold_pred[valid_index, :] = model.predict(x_te).reshape(-1, 1)
    # 폴드 세트 내에서 학습된 모델을 통해 입력된 원본 테스트 데이터를 예측한 후 데이터 저장
    test_pred[:, folder_counter] = model.predict(x_test_n)

  # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
  test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

  # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터
  # test_pred_mean은 최종 메타 모델이 사용하는 테스트 데이터 
  return train_fold_pred, test_pred_mean


In [21]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, x_train, y_train, x_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, x_train, y_train, x_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, x_train, y_train, x_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, x_train, y_train, x_test, 7)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [22]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('스태킹 학습 피처 데이터 shape: ', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 shape: ', Stack_final_X_test.shape)

스태킹 학습 피처 데이터 shape:  (455, 4) 스태킹 테스트 피처 데이터 shape:  (114, 4)


In [23]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9737
